<img src="https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/media/logo/newebac_logo_black_half.png" alt="ebac-logo">

---

# EDA e análise de crédito com SQL

João Lucas Carrijo Vieira

https://www.linkedin.com/in/joaolucascv07/

https://github.com/joaolucascv

---

# **Tópicos**

<ol type="1">
  <li>Introdução;</li>
  <li>Exploração dos Dados;</li>
  <li>Análise dos Dados;</li>
  <li>Storytelling.</li>
</ol>

---

## 1\. Introdução


Esse projeto faz parte do curso SQL para análise de dados da EBAC.

O dataset utilizado foi disponibilizado pelo professor André Luiz e pode ser acessado integralmente no link abaixo.

Utilizei o banco de dados AWS S3 para criar o bucket.

A tabela e todas as querys de análise foram feitas no AWS Athena e os resultados foram carregados como imagem nesse notebook, conforme link abaixo.

 - **Dataset**: GitHub André Peres ([link](https://github.com/andre-marcos-perez/ebac-course-utils/tree/main/dataset)).

 - **Resultado das Querys**: Imagens no GitHub ([link](https://github.com/joaolucascv/sqlebac-img.git)).

### **1.1. Dados**

Os dados são de clientes de um banco que disponibiliza cartões de crédito e contam com as seguintes colunas:  

- **idade** = idade do cliente
- **sexo** = sexo do cliente (F ou M)
- **dependentes** = número de dependentes do cliente
- **escolaridade** = nível de escolaridade do clientes
- **estado civil** = estado civil dos clientes
- **salario_anual** = faixa salarial do cliente
- **tipo_cartao** = tipo de cartao do cliente
- **qtd_produtos** = quantidade de produtos comprados nos últimos 12 meses
- **iteracoes_12m** = quantidade de iterações/transacoes nos ultimos 12 meses
- **meses_inativo_12m** = quantidade de meses que o cliente ficou inativo
- **limite_credito** = limite de credito do cliente
- **valor_transacoes_12m** = valor das transações dos ultimos 12 meses
- **qtd_transacoes_12m** = quantidade de transacoes dos ultimos 12 meses

Query de criação da tabela:

CREATE EXTERNAL TABLE IF NOT EXISTS default.credito (  
  `idade` int,  
  `sexo` string,  
  `dependentes` int,  
  `escolaridade` string,  
  `estado_civil` string,  
  `salario_anual` string,  
  `tipo_cartao` string,  
  `qtd_produtos` bigint,  
  `iteracoes_12m` int,  
  `meses_inativo_12m` int,  
  `limite_credito` float,  
  `valor_transacoes_12m` float,
  `qtd_transacoes_12m` int  
  )

ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe'  
WITH SERDEPROPERTIES (  
  'serialization.format' = ',',  
  'field.delim' = ','  
  ) LOCATION 's3://ebac-joaolucas-projeto/'  

TBLPROPERTIES ('has_encrypted_data'='false');

## 2\. Exploração de Dados

Nessa fase vamos entender nossa matéria-prima. Vamos explorar a quantidade de dados, o tipo dos dados e a qualidade deles.

**Qual a quantidade de informações temos na nossa base de dados?**

Query: SELECT count(*) FROM credito

![](https://raw.githubusercontent.com/joaolucascv/sqlebac-img/refs/heads/main/count.png)

Temos 2564 linhas na base de dados.


**Vamos visualizar as 10 primeiras linhas para entender como são os dados:**

Query: SELECT * FROM credito LIMIT 10;

![](https://github.com/joaolucascv/sqlebac-img/blob/main/limit10.png?raw=true)


Podemos observar que a base de dados está dividida em 11 colunas conforme o item 1.1 desse projeto.

**Qual o tipo de cada coluna na nossa base de dados?**

Query: DESCRIBE credito

![](https://raw.githubusercontent.com/joaolucascv/sqlebac-img/refs/heads/main/describe.png)


Vamos trabalhar com as variáveis que não são numéricas.


A segunda fase da exploração será mais focada nas variáveis categóricas como: escolaridade, estado civil, salário anual e tipo de cartão.

**Tipos de Escolaridade**

Query: SELECT DISTINCT escolaridade FROM credito

![](https://github.com/joaolucascv/sqlebac-img/blob/main/escolaridade.png?raw=true)


**Tipos de Estado civil**

Query: SELECT DISTINCT estado_civil FROM credito

![](https://github.com/joaolucascv/sqlebac-img/blob/main/estado_civil.png?raw=true)

As variáveis escolaridade e estado civil possuem valores nulos que vamos tratar mais adiante nesse projeto.



**Faixas do salário anual**

Query: SELECT DISTINCT salario_anual FROM credito

![](https://github.com/joaolucascv/sqlebac-img/blob/main/salario_anual.png?raw=true)

Na base de dados não temos o salário específico de cada cliente, porém temos a faixa salarial que ele se encontra.



**Tipos de cartão**

Query: SELECT DISTINCT tipo_cartao FROM credito

![](https://github.com/joaolucascv/sqlebac-img/blob/main/cartao.png?raw=true)

Nessa variável não temos valores nulos.



## 3\. Análise dos Dados

Uma vez que exploramos o dados e buscamos entender quais são as informações que temos no nosso banco de dados, podemos analisar as informações para buscar entender o que está acontecendo no banco de dados.

1. Quantos clientes temos de cada faixa salarial?
1. Quantos clientes são homens e quantos são mulheres?
1. Qual a idade dos clientes?
1. A idade impacta no uso do cartão de crédito?
1. Qual a maior e menor transação dos clientes?
1. Quais as características dos clientes que mais usaram o cartão nos últimos 12 meses? E dos que menos usaram?
1. O estado civil e o número de dependentes impacta no uso do cartão?

**Quantos clientes temos de cada faixa salarial?**

Query: SELECT count(*), salario_anual from credito  
group by salario_anual

![](https://github.com/joaolucascv/sqlebac-img/blob/main/faixa_salarial.png?raw=true)
  
A faixa salarial que possui maior número de clientes desse banco é dos que recebem menos do que $ 40K por ano.


**Quantos clientes são homens e quantos são mulheres?**

Query: SELECT count(*), sexo from credito  
group by sexo

![](https://github.com/joaolucascv/sqlebac-img/blob/main/faixa_sexo.png?raw=true)

![](https://github.com/joaolucascv/sqlebac-img/blob/main/grafico-qde-sexo.png?raw=true)


A base de dados está dividida proporcionalmente em 60% de homens e 40% de mulheres aproximadamente. Do csv extraído dos dados gerei um gráfico em pizza para melhor visualização no excel.



**Qual a idade dos clientes?**

Query: SELECT avg(idade) as media_idade, min(idade) as min_idade, max(idade) as max_idade, sexo from credito  
group by sexo

![](https://github.com/joaolucascv/sqlebac-img/blob/main/med-idade.png?raw=true)

A idade dos clientes varia entre 26 e 73 anos, sendo a média 46 anos de idade.


**A idade impacta no uso do cartão de crédito?**

Query: SELECT CAST(avg(valor_transacoes_12m) AS INT) as media_valor_gasto, idade from credito  
group by idade

![](https://github.com/joaolucascv/sqlebac-img/blob/main/Idade%20e%20uso%20do%20cart%C3%A3o.png?raw=true)

Usei a função CAST para gerar um número inteiro na média do valor das transações dos últimos 12 meses e com o resultado csv criei um gráfico de barras. Podemos observar que a medida que a idade dos clientes aumenta diminui o valor da média do uso do cartão de crédito.


**Qual a maior e menor transação dos clientes em 12 meses?**

Query: SELECT min(valor_transacoes_12m) as transacao_minima, max(valor_transacoes_12m) as transacao_maxima from credito

![](https://github.com/joaolucascv/sqlebac-img/blob/main/transacoes%20min-max.png?raw=true)

As transações variam de 510,16 até 4776,58 no período de 12 meses.



**Quais as características dos clientes que mais usaram o cartão nos últimos 12 meses? E dos que menos usaram?**

Query 1 : SELECT max(valor_transacoes_12m) as maiores_transacoes, escolaridade, tipo_cartao, sexo from credito  
where escolaridade != 'na' and tipo_cartao != 'na'  
group by escolaridade, tipo_cartao, sexo  
order by maiores_transacoes desc  
limit 10

![](https://github.com/joaolucascv/sqlebac-img/blob/main/maiores%2010%20transacoes.png?raw=true)


Query 2: SELECT max(valor_transacoes_12m) as menores_transacoes, escolaridade, tipo_cartao, sexo from credito  
where escolaridade != 'na' and tipo_cartao != 'na'  
group by escolaridade, tipo_cartao, sexo  
order by menores_transacoes asc  
limit 10

![](https://github.com/joaolucascv/sqlebac-img/blob/main/menores%2010%20transacoes.png?raw=true)


Não conseguimos a partir dos dados definir um padrão de relação entre o sexo, a escolaridade e o uso do cartão. Já no tipo do cartão podemos observar que quem mais usa são pessoas que possuem o cartão blue, enquanto cartões platinum, gold e silver estão associados às menores transações.



**O estado civil e o número de dependentes impactam no uso do cartão?**

Query 1: SELECT max(valor_transacoes_12m) as maiores_transacoes, estado_civil, dependentes from credito  
where estado_civil != 'na'  
group by estado_civil, dependentes  
order by maiores_transacoes desc  
limit 10

![](https://github.com/joaolucascv/sqlebac-img/blob/main/maiores%2010%20transacoes%20estado%20civil%20e%20dep.png?raw=true)


Query 2: SELECT min(valor_transacoes_12m) as menores_transacoes, estado_civil, dependentes from credito  
where estado_civil != 'na'  
group by estado_civil, dependentes  
order by menores_transacoes desc  
limit 10

![](https://github.com/joaolucascv/sqlebac-img/blob/main/menores%2010%20transacoes%20estado%20civil%20e%20dep.png?raw=true)


Podemos observar uma tendência de que indivíduos solteiros e casados realizam transações maiores, especialmente aqueles com 3 ou 4 dependentes. Indivíduos divorciados tendem a ter transações menores, mas ainda significativas. Por fim,o número de dependentes pode influenciar o valor das transações, com mais dependentes potencialmente levando a transações maiores.

## 4\. Storytelling

**Comportamento e consumo com cartão de crédito**

Em um banco com uma base de clientes diversificada, a maioria dos clientes pertence à faixa salarial de menos de **$ 40K por ano**. Essa é uma população que, embora não tenha altos rendimentos, demonstra um uso significativo de cartões de crédito, especialmente em transações do dia a dia. A base de dados é composta por 60% de homens e 40% de mulheres, com idades variando **entre 26 e 73 anos, e uma média de 46 anos**. Essa distribuição etária e de gênero reflete uma amostra equilibrada, mas com nuances interessantes no comportamento financeiro.

Ao analisar o uso do cartão de crédito, observamos que a média de transações nos últimos 12 meses diminui à medida que a idade avança. Isso sugere que clientes **mais jovens tendem a utilizar mais o cartão**, possivelmente para cobrir despesas cotidianas, enquanto os clientes mais velhos podem ter um perfil de consumo mais conservador ou já estabilizado financeiramente.

Um dado curioso é que o tipo de cartão **mais utilizado é o cartão Blue**, que geralmente é associado a clientes com menor renda ou que buscam benefícios básicos. Enquanto isso, cartões premium como Platinum, Gold e Silver são menos utilizados, o que pode indicar que os clientes com maior poder aquisitivo preferem outras formas de pagamento ou não veem valor suficiente nos benefícios oferecidos por esses cartões.

Outro aspecto relevante é o **estado civil e o número de dependentes**. Clientes solteiros e casados tendem a realizar transações maiores, especialmente aqueles com 3 ou 4 dependentes. Isso pode estar relacionado a despesas familiares, como educação, saúde e lazer. Por outro lado, indivíduos divorciados tendem a ter transações menores, mas ainda significativas, possivelmente devido a uma reorganização financeira pós-divórcio. O número de dependentes parece ser um fator importante, já que mais dependentes estão associados a transações maiores, refletindo o custo de vida de famílias maiores.

Com base nesses dados, é possível traçar algumas estratégias para o departamento de marketing:

**Foco no Cartão Blue**:

Como o cartão Blue é o mais utilizado, o marketing pode destacar seus benefícios, como cashback, descontos em estabelecimentos parceiros e programas de pontos. Campanhas que reforcem a praticidade e o valor agregado desse cartão podem atrair ainda mais clientes da faixa salarial abaixo de $ 40K.

**Segmentação por Idade**:

Para clientes mais jovens, que tendem a usar mais o cartão, crie campanhas que incentivem o uso responsável, como programas de educação financeira ou ofertas especiais para compras online.

Para clientes mais velhos, que usam menos o cartão, ofereça benefícios que atendam às suas necessidades, como descontos em farmácias, planos de saúde ou seguros.

**Aproveitar o Perfil Familiar**:

Como clientes casados e com dependentes tendem a realizar transações maiores, crie campanhas que ofereçam benefícios para famílias, como descontos em supermercados, escolas ou atividades de lazer.

Ofereça cartões com limites mais altos ou programas de recompensas para famílias com 3 ou mais dependentes.

**Engajamento de Divorciados**:

Para clientes divorciados, que têm transações menores, mas ainda significativas, ofereça produtos financeiros que ajudem na reorganização de suas finanças, como empréstimos com taxas especiais ou programas de recompensas para gastos essenciais.

**Promoção de Cartões Premium**:

Para aumentar o uso de cartões Platinum, Gold e Silver, o marketing pode focar em benefícios exclusivos, como acesso a salas VIP, seguros gratuitos ou programas de milhagem. Destaque o status e a exclusividade associados a esses cartões.

**Uso de Dados para Personalização**:

Utilize os dados demográficos e comportamentais para criar campanhas altamente personalizadas. Por exemplo, envie ofertas específicas para homens solteiros com 2 dependentes ou mulheres casadas com 3 dependentes.